In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymc3 as pm
import arviz as az

WARNING (theano.link.c.cmodule): install mkl with `conda install mkl-service`: No module named 'mkl'
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [11]:
model = pm.Model()

with model:
    cutremur = pm.Bernoulli('C', 0.0005)
    incendiu = pm.Deterministic('I', pm.math.switch(cutremur, 0.03, 0.01))
#     este cutremur si incendiu: 98%
#     este cutremur si nu este incendiu:  2%
#     fara cutremur si este incendiu: 95%
#     fara cutremur si fara incendiu: 0.01%
# cumparat produs <=> alarma de incendiu declansata
# sa fi avut nevoie urgenta de produs  <=> sa fi avut loc un cutremur
    false_alarm_p = pm.Deterministic('FA_p', pm.math.switch(cutremur, pm.math.switch(incendiu, 0.98, 0.02), pm.math.switch(incendiu, 0.95, 0.0001)))
    false_alarm = pm.Bernoulli('FA', p= false_alarm_p, observed=1)
    trace = pm.sample(20000)

/srv/conda/envs/notebook/lib/python3.7/site-packages/deprecat/classic.py:215: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  return wrapped_(*args_, **kwargs_)
Multiprocess sampling (4 chains in 4 jobs)
BinaryGibbsMetropolis: [C]


Sampling 4 chains for 1_000 tune and 20_000 draw iterations (4_000 + 80_000 draws total) took 37 seconds.


In [12]:
dictionary = {
              'cutremur': trace['C'].tolist(),
              'incendiu': trace['I'].tolist()
              }
df = pd.DataFrame(dictionary)

In [13]:
p_cutremur = df[(df['cutremur'] == 1)].shape[0] / df.shape[0]
print(p_cutremur)

0.0006125
